<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#适配器模式:-一个简单的例子" data-toc-modified-id="适配器模式:-一个简单的例子-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>适配器模式: 一个简单的例子</a></span></li></ul></li></ul></li><li><span><a href="#Adapter-Pattern" data-toc-modified-id="Adapter-Pattern-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adapter Pattern</a></span></li></ul></div>

###### 适配器模式: 一个简单的例子

适配器模式通过引入间接层来实现不兼容接口之间的适配。

开闭原则：适配器模式和OOP中的开闭原则关系密切，开闭原则强调对扩展开放，对修改关闭。通过适配器模式我们可以通过创建适配器模式在不修改原有类代码的情况下实现新的功能。

In [3]:
class Computer(object):

    def __init__(self, name):

        self.name = name

    def __str__(self):

        return f'the {self.name} computer'

    def execute(self):

        print('Execute a program.')


class Synthesizer(object):

    def __init__(self, name):

        self.name = name

    def __str__(self):

        return f'the {self.name} synthesizer'

    def play(self):

        print('Playing a song.')


class Human(object):
    
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'the {} human'.format(self.name)

    def speak(self):
        print('Saying hello.')

In [5]:
class Adapter(object):
    
    def __init__(self, obj, adapted_method:dict):
        """不使用继承, 使用 __dict__ 实现适配器模式.
        """
        self.obj = obj
        self.__dict__.update(adapted_method)
    
    def __str__(self):
        
        return str(self.obj)

In [7]:
# 使用示例
objs = [Computer('ASUS'), ]

synth = Synthesizer('MOOG')
objs.append(Adapter(obj=synth, adapted_method=dict(execute=synth.play)))

human = Human('WNN')
objs.append(Adapter(obj=human, adapted_method=dict(execute=human.speak)))

In [8]:
for obj in objs:
    # 用统一的 execute 接口适配不同对象的方法, 这样
    # 在无需修改源码的情况下完成了不同方法的适配
    print(obj)
    obj.execute()

the ASUS computer
Execute a program.
the MOOG synthesizer
Playing a song.
the WNN human
Saying hello.


#### Adapter Pattern

In [9]:
class Page(object):
    """用来渲染页面的类, Page class
    不知道 renderer 属性的类的定义,
    它只需要调用三个方法: header, paragraph 和 footer

    """

    def __init__(self, title, renderer):

        if not isinstance(renderer, Renderer):
            raise TypeError(
                f'Excepted object of type Renderer, found {type(renderer).__name__}')

        self.title = title
        self.renderer = renderer
        self.paragraphs = []

    def add_paragraph(self, paragraph):

        self.paragraphs.append(paragraph)

    def render(self):

        self.renderer.header(self.title)
        for paragraph in self.paragraphs:
            self.renderer.paragraph(paragraph)
        self.renderer.footer()

上面的实现带来的一个问题是: 我们所有的 renderer 实例都必须是 Renderer或者其子类的实例.
我们可以用 \_\_subclasshook\_\_ 解决这个问题

In [21]:
import abc
import collections


class Renderer(metaclass=abc.ABCMeta):
    
    @classmethod
    def __subclasshook__(class_, subclass_):
        """__subclasshook__ 函数在 isinstance() 调用的
        时候被调用, 用来判断:
        if the object it is given as its first argument is a subclass of the class 
        (or any of the tuple of classes) it is passed as its second argument.
        
        Parameters
        ----------
        classs_: isinstance 的第二个参数, class or tuple of cls
        subclass_: class_ 子类的 instance, 注意如果不是 class_ 的子类的实例是不会调用 __subclasshook_ 的.
        """
        print('Calling __subclasshook__.')
        print('class_ is: ', class_)
        print('subclass_ is: ', subclass_)
        if class_ is Renderer: # 只有 isinstance(instance, Renderer)的时候才会被执行下面的代码, subcalss的话不会执行
            attributes = collections.ChainMap(*( cls_.__dict__ for cls_ in subclass_.__mro__))
            methods = ('header', 'paragraph', 'footer')
            if all(method in attributes for method in methods):
                return True
        return NotImplemented

In [22]:
class test_renderer(Renderer):
    def header(self): pass
    def paragraph(self): pass
    def footer(self): pass

r = test_renderer()

isinstance(r, Renderer)

Calling __subclasshook__.
class_ is:  <class '__main__.Renderer'>
subclass_ is:  <class '__main__.test_renderer'>


True